Uses ConvNeXT instead of ResNet50 for the embeddings of the aerial images

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import torch
from torchvision import transforms
from torchvision.models import convnext_large, ConvNeXt_Large_Weights
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from sklearn.decomposition import PCA

In [ ]:
class RegionsDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform
        self.missing_images = []

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = Path(self.root_dir) / f"{self.dataframe.index[idx]}.jpg"
        if not img_name.exists():
            self.missing_images.append(self.dataframe.index[idx])
            return None  # Skip missing images

        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

In [ ]:
# Step 1: Prepare regions
regions_buffered_gdf = gpd.read_file("selected_regions_buffered_9.geojson")
regions_buffered_gdf = regions_buffered_gdf.to_crs(epsg=28992)
regions_buffered_gdf.set_index('region_id', inplace=True)
bbox = regions_buffered_gdf.geometry.bounds
regions_buffered_gdf['minx'] = bbox['minx']
regions_buffered_gdf['miny'] = bbox['miny']
regions_buffered_gdf['maxx'] = bbox['maxx']
regions_buffered_gdf['maxy'] = bbox['maxy']

# Verify if regions_buffered_gdf index is contained within the image folder
assert regions_buffered_gdf.index.isin([f.stem for f in Path("../images").glob("*.jpg")]).all()

In [ ]:
# Step 2: Load the Pretrained ConvNeXt Large Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'device {device}')
model = convnext_large(weights=ConvNeXt_Large_Weights.DEFAULT)
model = torch.nn.Sequential(*(list(model.children())[:-1]))
model = model.to(device)
model.eval()  # Set the model to evaluation mode

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:
# Step 3: Infer Embeddings for Each Region
# Initialize the dataset and dataloader
dataset = RegionsDataset(dataframe=regions_buffered_gdf, root_dir="../images", transform=transform)
dataloader = DataLoader(dataset, batch_size=128, shuffle=False)

# Extract embeddings
embeddings = []
missing_images_info = []

with torch.no_grad():
    for images in tqdm(dataloader):
        if images is None:  # Ensure there are no None values
            continue
        images = images.to(device)
        outputs = model(images)
        embeddings.extend(outputs.cpu().numpy())  # Collect embeddings

# Log missing images after processing
if dataset.missing_images:
    missing_images_info = regions_buffered_gdf.loc[dataset.missing_images]

# Flatten the embeddings and prepare for DataFrame
flattened_embeddings = np.vstack(embeddings).reshape(len(embeddings), -1)

In [ ]:
# Step 4: Create the DataFrame
aerial_embeddings_df = pd.DataFrame(flattened_embeddings, index=regions_buffered_gdf.index)
aerial_embeddings_df.columns = [f"emb_{i}" for i in range(flattened_embeddings.shape[1])]

# Apply Principal Component Analysis to reduce dimensionality
pca = PCA(n_components=512)
reduced_embeddings = pca.fit_transform(aerial_embeddings_df)

# Create a new DataFrame for the reduced embeddings
reduced_embeddings_df = pd.DataFrame(reduced_embeddings, index=aerial_embeddings_df.index)
reduced_embeddings_df.columns = [f'PCA_{i}' for i in range(512)]



In [ ]:
# Step 5: Export the embeddings to a CSV file
aerial_embeddings_df.to_csv('embeddings_aerial_9_convnext.csv')

In [ ]:
# from Plotting import pca_plot, cluster_plot
# import warnings
# warnings.filterwarnings("ignore")
# cluster_plot(reduced_embeddings_df, regions_buffered_gdf, 14)

In [ ]:
# import embeddings aerial resnet50
aerial_embeddings_resnet50_df = pd.read_csv('embeddings_aerial_9_resnet50.csv', index_col=0)

In [ ]:
cluster_plot(aerial_embeddings_resnet50_df, regions_buffered_gdf, 14)